In [1]:
import pandas as pd
import numpy as np
from linearmodels import PanelOLS

import multiprocessing as mp
import multiprocessing.dummy as mpd

from scipy.optimize import newton_krylov, anderson, broyden1, broyden2

/bbkinghome/tadej/miniconda3/envs/main/lib/python3.7/site-packages/linearmodels/panel/data.py:10: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import (Categorical, DataFrame, Index, MultiIndex, Panel, Series,


# Read in data

Here I read in data. The important variables are:
- `exporter`, `importer`, `industry`: These ones are self explanatory -- correspond to $i, j, k$, respectively
- `exports`: Also self explanatory, corresponds to $x_{ij}^k$
- `ipr`: This is equivalent to $1-\pi_{ii}^k$ from the paper
- `Gro_PPP_SO_97`: Producer price index based on gross output -- the inverse of this is $\tilde{z}_{i}^k$ 
- `rdexpend`: R&D expenditures

So I will create these new variables:
- `z` (equal to $\ln(\tilde{z}_{i}^k)$)
- `x1` (equal to $ln({x}_{ij}^k)$)
- `x` (equal to $\ln(\tilde{x}_{ij}^k)$)
- `d_ij`: importer-exporter fixed effects ($\delta_{ij}$)
- `d_jk`: importer-industry fixed effects ($\delta_{jk}$)
- `rd`: log of R&D expenditures

In [2]:
data = pd.read_stata('cdk2010.dta')
data['year'] = 1997
data = data.set_index(['exporter', 'year'], drop = False)

# Add 1 to exports bc of log
data['exports'] = data['exports'] + 1

In [3]:
data['x1'] = np.log(data['exports'])
data['x'] = data['x1'] - np.log(1-data['ipr']/100)
data['z'] = -np.log(data['Gro_PPP_SO_97'])

data['d_ij'] = data['exporter'] + '_' + data['importer']
data['d_jk'] = data['importer'] + '_' + data['industry']

data['rd'] = np.log(data['rdexpend'])

/bbkinghome/tadej/miniconda3/envs/main/lib/python3.7/site-packages/pandas/core/series.py:853: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


# OLS

In [4]:
PanelOLS(dependent = data['x'], exog = data['z'], other_effects = data[['d_ij', 'd_jk']]).fit()

Dep. Variable:,x,R-squared:,0.0241
Estimator:,PanelOLS,R-squared (Between):,-0.2215
No. Observations:,5733,R-squared (Within):,0.0000
Date:,"Fri, Oct 18 2019",R-squared (Overall):,-0.2145
Time:,01:34:57,Log-likelihood,-1.096e+04
Cov. Estimator:,Unadjusted,,
,,F-statistic:,124.03
Entities:,21,P-value,0.0000
Avg Obs:,273.00,Distribution:,"F(1,5019)"
Min Obs:,273.00,,
Max Obs:,273.00,F-statistic (robust):,124.03


In [5]:
PanelOLS(dependent = data['x1'], exog = data['z'], other_effects = data[['d_ij', 'd_jk']]).fit()

Dep. Variable:,x1,R-squared:,0.0311
Estimator:,PanelOLS,R-squared (Between):,-0.2731
No. Observations:,5733,R-squared (Within):,0.0000
Date:,"Fri, Oct 18 2019",R-squared (Overall):,-0.2642
Time:,01:34:57,Log-likelihood,-1.099e+04
Cov. Estimator:,Unadjusted,,
,,F-statistic:,161.14
Entities:,21,P-value,0.0000
Avg Obs:,273.00,Distribution:,"F(1,5019)"
Min Obs:,273.00,,
Max Obs:,273.00,F-statistic (robust):,161.14


# IV

Unfortunately I have to do IV in stata, as only it can drop some annoying variables. Below is the code I use to construct the variables and run the regressions

```
use cdk2010

gen z = -ln(Gro_PPP_SO_97)

gen x1 = ln(exports)
replace ipr = ipr/100
gen x = x1 - ln(1-ipr)
gen rd = log(rdexpend)

gen d_ij = importer + "_" + exporter
gen d_jk = importer + "_" + industry

ivreghdfe x (z=rd), absorb(d_ij d_jk)
ivreghdfe x1 (z=rd), absorb(d_ij d_jk)
```

Bellow you can find the output for the last 2 commands

```
IV (2SLS) estimation
--------------------

Estimates efficient for homoskedasticity only
Statistics consistent for homoskedasticity only

                                                      Number of obs =     5649
                                                      F(  1,  4955) =    44.95
                                                      Prob > F      =   0.0000
Total (centered) SS     =   14677.7366                Centered R2   =  -0.2364
Total (uncentered) SS   =   14677.7366                Uncentered R2 =  -0.2364
Residual SS             =  18147.03231                Root MSE      =    1.914

------------------------------------------------------------------------------
           x |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
           z |   5.814509   .8672592     6.70   0.000     4.114297    7.514722
------------------------------------------------------------------------------
Underidentification test (Anderson canon. corr. LM statistic):         138.034
                                                   Chi-sq(1) P-val =    0.0000
------------------------------------------------------------------------------
Weak identification test (Cragg-Donald Wald F statistic):              124.109
Stock-Yogo weak ID test critical values: 10% maximal IV size             16.38
                                         15% maximal IV size              8.96
                                         20% maximal IV size              6.66
                                         25% maximal IV size              5.53
Source: Stock-Yogo (2005).  Reproduced by permission.
------------------------------------------------------------------------------
Sargan statistic (overidentification test of all instruments):           0.000
                                                 (equation exactly identified)
------------------------------------------------------------------------------
Instrumented:         z
Excluded instruments: rd
Partialled-out:       _cons
                      nb: total SS, model F and R2s are after partialling-out;
                          any small-sample adjustments include partialled-out
                          variables in regressor count K
------------------------------------------------------------------------------

Absorbed degrees of freedom:
-----------------------------------------------------+
 Absorbed FE | Categories  - Redundant  = Num. Coefs |
-------------+---------------------------------------|
        d_ij |       441           0         441     |
        d_jk |       273          21         252     |
-----------------------------------------------------+

```

```
IV (2SLS) estimation
--------------------

Estimates efficient for homoskedasticity only
Statistics consistent for homoskedasticity only

                                                      Number of obs =     5649
                                                      F(  1,  4955) =    89.96
                                                      Prob > F      =   0.0000
Total (centered) SS     =  14797.59137                Centered R2   =  -1.0273
Total (uncentered) SS   =  14797.59137                Uncentered R2 =  -1.0273
Residual SS             =  29998.46752                Root MSE      =    2.461

------------------------------------------------------------------------------
          x1 |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
           z |   10.57593   1.115053     9.48   0.000     8.389934    12.76193
------------------------------------------------------------------------------
Underidentification test (Anderson canon. corr. LM statistic):         138.034
                                                   Chi-sq(1) P-val =    0.0000
------------------------------------------------------------------------------
Weak identification test (Cragg-Donald Wald F statistic):              124.109
Stock-Yogo weak ID test critical values: 10% maximal IV size             16.38
                                         15% maximal IV size              8.96
                                         20% maximal IV size              6.66
                                         25% maximal IV size              5.53
Source: Stock-Yogo (2005).  Reproduced by permission.
------------------------------------------------------------------------------
Sargan statistic (overidentification test of all instruments):           0.000
                                                 (equation exactly identified)
------------------------------------------------------------------------------
Instrumented:         z
Excluded instruments: rd
Partialled-out:       _cons
                      nb: total SS, model F and R2s are after partialling-out;
                          any small-sample adjustments include partialled-out
                          variables in regressor count K
------------------------------------------------------------------------------

Absorbed degrees of freedom:
-----------------------------------------------------+
 Absorbed FE | Categories  - Redundant  = Num. Coefs |
-------------+---------------------------------------|
        d_ij |       441           0         441     |
        d_jk |       273          21         252     |
-----------------------------------------------------+
```

# Shock effect estimation

In [6]:
data = data.reset_index(drop = True)
t = 6.35

# Get all countries and industries
countries = data['exporter'].unique()
industries = data['industry'].unique()

# Calculate income, total exports and total consumption in sector
exports = data.groupby(['exporter', 'importer', 'industry'])['exports'].sum()
GDP = exports.groupby(['importer']).sum()
#GDP.index = GDP.index.rename('exporter')

#total_consumption = exports.groupby(['importer']).sum()
consumption = exports.groupby(['importer', 'industry']).sum()

# Obtain expenditure shapres
alpha = consumption/GDP

# Obtain import shares
pi = (exports/consumption).reorder_levels([2,0,1])

# Trade imbalance
#total_consumption.index = total_consumption.index.rename('exporter')
imbalance = (exports.groupby(['exporter']).sum() - GDP)/GDP.sum()

# Reindex for easier calculations later
consumption.index = consumption.index.rename(['exporter', 'industry'])
GDP.index = GDP.index.rename('exporter')

In [14]:
# Changes (hat w just the provisional first guess)
hat_d = pd.Series(1, index = exports.index)
hat_w = pd.Series(1.0, index = GDP.index)
hat_z = pd.Series(1.0, index = consumption.index)

# 80% productivity increase in Japan
hat_z['JPN'] = 1.8

In [30]:
cnt_dict = {}
for i, cnt in enumerate(countries):
    cnt_dict[cnt] = i

# Calculates the denominator in equation (1)
def denominator(j, k, hat_w, hat_d, hat_z, pi, t):
    
    # We're looping over exporters, so column will be 'exporter'
    pi_t = pi[:, j, k]
    hat_d_t = hat_d[:, j, k] 
    hat_z_t = hat_z[:, k]
    
    return (pi_t * np.power(hat_w * hat_d_t / hat_z_t, -t)).sum()

# Calculates the item inside the sum
def sum_item(i, j, k, hat_w, hat_d, hat_z, pi, GDP, alpha, t):
    upper = pi[i, j, k] * np.power(hat_w[cnt_dict[i]] * hat_d[i, j, k] / hat_z[i, k], -t)
    lower = denominator(j, k, hat_w, hat_d, hat_z, pi, t)
    right = alpha[j, k] * hat_w[cnt_dict[j]] * GDP[j]

    return (upper/lower) * right

# Calculates the LHS of the equation
def LHS(i, hat_w, hat_d, hat_z, pi, GDP, alpha, t):
    s = 0
    
    for j in countries:
        for k in industries:
            s += sum_item(i, j, k, hat_w, hat_d, hat_z, pi, GDP, alpha, t)
            
    return s

#Calculates the RHS of the equation
def RHS(i, hat_w, GDP, imbalance):
    return hat_w[cnt_dict[i]] * GDP[i] + imbalance[i] * (hat_w*GDP).sum()

def R(args):
    i = args[0]
    hat_w = args[1]
    
    return LHS(i, hat_w, hat_d, hat_z, pi, GDP, alpha, t) - RHS(i, hat_w, GDP, imbalance)

# The function for the optimizer
def F(hat_w):
    loopy = [(i, hat_w) for i in countries]

    with mp.Pool(mp.cpu_count()) as pool:
        r = np.array(pool.map(R, loopy))/GDP.sum()
    
    return r

In [31]:
# Solve it
hat_w = pd.Series(anderson(F, hat_w), index = hat_w.index)

In [32]:
# Welfare
def welfare_i(i, pi, hat_w, hat_d, hat_z, alpha, t):
    s = 1
    for k in industries:
        s = s*denominator(i, k, hat_w, hat_d, hat_z, pi, t)
        
    return hat_w[i]/np.power(s, -alpha[i,k]/t)

welfare = pd.Series([welfare_i(i, pi, hat_w, hat_d, hat_z, alpha, t) for i in countries], index = hat_w.index)

In [49]:
table = pd.concat([hat_w.rename('Change in w'), welfare.rename('Change in welfare')], axis = 1)
table.index.name = None
table = (table -1)*100

In [51]:
print(table.round(2).to_latex())

\begin{tabular}{lrr}
\toprule
{} &  Change in w &  Change in welfare \\
\midrule
AUS     &        -2.12 &              -1.36 \\
BEL-LUX &        -3.82 &              -2.66 \\
CZE     &        -3.36 &              -1.87 \\
DNK     &        -3.41 &              -1.98 \\
ESP     &        -3.39 &              -1.92 \\
FIN     &        -3.86 &              -2.83 \\
FRA     &        -3.53 &              -2.02 \\
GER     &        -3.70 &              -2.24 \\
GRC     &        -1.89 &              -0.66 \\
HUN     &        -3.53 &              -2.88 \\
IRL     &        -3.94 &              -2.10 \\
ITA     &        -3.78 &              -2.48 \\
JPN     &        66.80 &              72.17 \\
KOR     &        -3.33 &              -2.66 \\
NLD     &        -3.38 &              -1.26 \\
POL     &        -2.86 &              -1.79 \\
PRT     &        -3.09 &              -1.47 \\
SVK     &        -3.42 &              -2.14 \\
SWE     &        -3.66 &              -2.56 \\
UK      &        -3.37 &  